In [1]:
%%capture
%pip install 'transformers<=4.34.0' openai torch scikit-learn
%%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Next, add your API key:

In [30]:
import os
from openai import AzureOpenAI
import openai

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

In [31]:
log_file_path = "C:/Users/kr4193/Desktop/Log_error_reporter/Prep_work/clean_Geiger_for_LLMs.log"
pytest_path = "C:/Users/kr4193/Downloads/evtstuff/atf/tests/print/clean_cutter.py"
# Read the log content from the file

file = open(pytest_path, "r")
content = file.read()
def read_log_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
documents = read_log_file(log_file_path)

In [34]:
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
document_index = embed_documents(documents, bert_model)
document_index[0].shape # shape of each vector in the index

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/53aa51172d142c89d9012cce15ae4d6cc0ca6895895114379cacb4fab128d9db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1739807314&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTgwNzMxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvYWxsLU1pbmlMTS1MNi12Mi81M2FhNTExNzJkMTQyYzg5ZDkwMTJjY2UxNWFlNGQ2Y2MwY2E2ODk1ODk1MTE0Mzc5Y2FjYjRmYWIxMjhkOWRiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=uaJfnKmtaCWTON~aELXN9IqQWSlpYgAVxRHrwpUXTD8nAKl27jgl0lt6enTlnFzqHYUgERH7lYP6BBTA8nVuQshS6axaXdZYNK6koQSRY3JoYQinMFhM-hMRT6mOSiDlaqxP2Vyb~QdtnPsqNm1z0rsl5Fgi-TmvHmvhH5RIQl~wEedL8ViYY7yCXI9sp3MTjkBkD00oXMHg8DJYBSgXIrhQPj38twBjGVdhsnNbpvVqTSf2MF4P8vAN92aJrsKPj~dOA8mP2PNDPakB304IIZPReAOr~kGZcYFDwuMEwCb

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def retrieve_documents(query_string, doc_index, docs, k=5, doc_model_name="BAAI/bge-base-en"):
  # embed the query string to obtain a query vector
  query_vector = embed_documents(
      [query_string],
      model_name="BAAI/bge-base-en"
  ).reshape(1, -1)

  # use the query vector to find the most similar document to the query
  similarity = cosine_similarity(query_vector, doc_index).flatten()

  # return the top k most similar docs
  # here, argsort assigns the indices that would order the similarities
  # from least similar to most similar. The [::-1] slice reverses this
  # to return most similar to least, and slices the top k of these
  return [docs[i] for i in np.argsort(similarity)[::-1][:k]]

In [ ]:
example_retrieved_docs = retrieve_documents(
    "Tell me about the Japanese lunar mission.",
    document_index,
    documents,
    k=3
)

example_retrieved_docs

NameError: name 'document_index' is not defined

In [ ]:
def create_augmented_prompt(query_string, docs):
  # concatenate the retrieved docs as context for the LLM
  # you could do other pre-processing here too
  context = "\n".join(docs)
  # define your prompt template
  prompt_template = """Here is some relevant information:
  {context}

  Q: {query}
  A:
  """
  # render the prompt template
  return prompt_template.format(context=context, query=query_string)

And you can see how this behaves with the following:

In [ ]:
example_augmented_prompt = create_augmented_prompt(
    "Tell me about the Japanese lunar mission.",
    example_retrieved_docs
)
example_augmented_prompt

'Here is some relevant information:\n  JAXA launched SLIM (Smart Lander for Investigating Moon) lunar lander (carrying a mini rover) and a space telescope (XRISM) on 6 September.\nISRO launched its third lunar mission Chandrayaan-3 on 14 July 2023 at 9:05 UTC; it consists of lander, rover and a propulsion module, and successfully landed in the south pole region of the Moon on 23 August 2023.\nRussian lunar lander Luna 25 was launched on 10 August 2023, 23:10 UTC, atop a Soyuz-2.1b rocket from the Vostochny Cosmodrome, it was the first Russian attempt to land a spacecraft on the Moon since the Soviet lander Luna 24 in 1974, it crashed on the Moon on 19 August after technical glitches.\n\n  Q: Tell me about the Japanese lunar mission.\n  A:\n  '

The last important piece is querying the LLM itself. This is simple enough. Here is another simple function to query the OpenAI GPT-4 API:

In [ ]:
import openai

def generate_response(query_string, model_name):
  response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant to classify the test logs."},
        {"role": "system", "content": "The fromat of tests is as follows Test suites -> Test modules -> Test cases , Test cases are the lowest level of granularity "},
        {"role": "system", "content": "Test Suites are started at the line Entering suites: <suitename> and ended at Suite <suitename> <result>"},
        {"role": "system", "content": "Test models are started at Execute module: <module_name>"},
        {"role": "system", "content": "Test cases are started at the line Running test: <test_name> and ended at line <test_name>: <result>"}, 
        {"role": "system", "content": "Always Ignore passed test cases and Suites"},
        {"role": "system", "content": "Always Identify and Classify the Failed tests into one of the 3 categories - Product issues, ATF Script Issues & Setup issues"},
        {"role": "system", "content": "Always display for failed test suites only - Possible fix - Suite - Module- Synopsis of the test Failed - test name -Failure category - Reason for categorisation - Assertion Failure  message "},
        #{"role": "system", "content": f"Here is the script that was used to run the test {content}"},
        {"role": "user", "content": f"Here is a log entry for analysis:\n{log_content}\n\nQuestion: {question}\n"},
    ]
)

  return response.choices[0].message.content

Once again, you can see how this behaves with:

In [ ]:
generate_response("Hello, world!", model_name="gpt-4")

'Hello! How can I assist you today?'

Okay, you've now seen all of the core components of creating using RAG to query an LLM. Time to bring it all together! 🚀

In [ ]:
def generate_rag_response(
    query_string,
    docs,
    doc_index,
    model_name="gpt-3.5-turbo",
    k=3
):

  # R: retrieve documents
  retrieved_docs = retrieve_documents(
      query_string, doc_index, documents
  )
  # A: create augmented prompt
  augmented_prompt = create_augmented_prompt(query_string, retrieved_docs)

  # G: generate response!
  generated_response = generate_response(augmented_prompt, model_name)
  return generated_response

Now generate a RAG response with:

In [35]:
generate_rag_response("Tell me about the status of the latest Indian lunar mission.", documents, document_index, model_name="gpt-4")

NameError: name 'generate_rag_response' is not defined

Up-to-date and accurate. Nice. Let's compare the RAG response to a 'raw' GPT-4 response: